In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import gpytorch
import math
import seaborn as sns
import random
from tqdm import trange, tqdm
from collections import Counter
from scipy.interpolate import splrep, interp1d
from sklearn import preprocessing
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.likelihoods import GaussianLikelihood, StudentTLikelihood
from gpytorch.models import ApproximateGP
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.means import ConstantMean
from gpytorch.mlls import ExactMarginalLogLikelihood, VariationalELBO
import matplotlib.ticker as ticker
import matplotlib.tri as tri
import matplotlib.font_manager as font_manager
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend
import time

class STP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=False
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)

        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class ExactGP(gpytorch.models.ExactGP):
    def __init__(self, x_train, y_train, likelihood):
        # build the model using the ExactGP model from gpytorch
        super(ExactGP, self).__init__(x_train, y_train, likelihood)

        # use a constant mean, this value can be learned from the dataset
        self.mean_module = gpytorch.means.ConstantMean()

        # automatically determine the number of dimensions for the ARD kernel
        num_dimensions = x_train.shape[1]

        # use a scaled Matern kernel, the ScaleKernel allows the kernel to learn a scale factor for the dataset
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5,ard_num_dims=num_dimensions))
            
        # set the number of outputs 
        self.num_outputs = 1

    def forward(self, x):
        # forward pass of the model

        # compute the mean and covariance of the model 
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)

        # return the MultivariateNormal distribution of the mean and covariance 
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class VariationalGP(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(VariationalGP, self).__init__(variational_strategy)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=inducing_points.size(1)))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
class TorchStandardScaler:
    def fit(self, x):
        x = x.clone()
        # calculate mean and std of the tensor
        self.mean = x.mean(0, keepdim=True)
        self.std = x.std(0, unbiased=False, keepdim=True)
    def transform(self, x):
        x = x.clone()
        # standardize the tensor
        x -= self.mean
        x /= (self.std + 1e-10)
        return x
    def fit_transform(self, x):
        # copy the tensor as to not modify the original 
        x = x.clone()
        # calculate mean and std of the tensor
        self.mean = x.mean(0, keepdim=True)
        self.std = x.std(0, unbiased=False, keepdim=True)
        # standardize the tensor
        x -= self.mean
        x /= (self.std + 1e-10)
        return x
    
class TorchNormalizer:
    def fit(self, x):
        # calculate the maximum value and the minimum value of the tensor
        self.max = torch.max(x, dim=0).values
        self.min = torch.min(x, dim=0).values

    def transform(self, x):
        # normalize the tensor
        return (x.clone() - self.min) / (self.max - self.min)

    def fit_transform(self, x):
        # calculate the maximum value and the minimum value of the tensor
        self.max = torch.max(x, dim=0).values
        self.min = torch.min(x, dim=0).values
        # normalize the tensor
        return (x.clone() - self.min) / (self.max - self.min)

# Expected Improvement
def EI(mean, std, best_observed, minimize=False):
    if minimize:
        improvement = best_observed - mean
    else:
        improvement = mean - best_observed
    
    z = improvement / std
    normal = torch.distributions.Normal(0, 1)
    
    ei = improvement * normal.cdf(z) + std * normal.log_prob(z).exp()
    return ei

def runMinSTP(seed):  
    print("Minimizing with STP") 
    set_seeds(seed)
    iterationSTP = [0]
    topSTP = [0]

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.05)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y > top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesSTP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesSTP = non_top_5_indices[indicesSTP]

    # Select the corresponding points from train_x and train_y
    train_xSTP = train_x[indicesSTP]
    train_ySTP = train_y[indicesSTP]

    print(f"Seed: {seed}")
    print(f"Initial points: {train_xSTP}")
        
    while topSTP[-1] < 100:
        iterationSTP.append(iterationSTP[-1] + 1)
        # standardize the initial inputs and outputs before
        train_xSTP = TorchStd.transform(train_xSTP)
        train_ySTP = TorchStandardScaler().fit_transform(train_ySTP).flatten()

        # normalize it  
        train_xSTP = TorchNorm.transform(train_xSTP)
        train_ySTP = TorchNormalizer().fit_transform(train_ySTP).flatten()
                                
        #def train_and_test_approximate_gp(objective_function_cls):
        model = STP(train_xSTP).to(dtype=torch.float64)
        #likelihood = gpytorch.likelihoods.GaussianLikelihood()
        likelihood = gpytorch.likelihoods.StudentTLikelihood()
        objective_function = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_ySTP.numel())
        # replace with gpytorch mll
        optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

        # Train
        model.train()
        #put it into training mode
        likelihood.train()

        for _ in range(50):
            output = model(train_xSTP)
            loss = -objective_function(output, train_ySTP)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Test
        model.eval()
        likelihood.eval()

        # works wtih the variational distribution
        with torch.no_grad(), gpytorch.settings.num_likelihood_samples(512):
            observed_pred = likelihood(model(TorchStd.transform(train_x))) # predict values for all candidates

        samples = observed_pred.sample()

        # use as inputs to EI
        meanSTP = samples.mean(dim=0)
        stdSTP = samples.std(dim=0)
        
        # Calculate the best observed value for EI
        best_observed_value = train_ySTP.min()
        
        # Select new points using EI
        acq_values = EI(meanSTP, stdSTP, best_observed_value, minimize=True)
        acq_values[indicesSTP] = -float('inf')
        indicesSTP = torch.cat([indicesSTP, acq_values.argmax().unsqueeze(0)])

        # Convert new_indices to a tensor
        # Add the new points to the training data
        train_xSTP = train_x[indicesSTP]
        train_ySTP = train_y[indicesSTP]
        topSTP.append(TopSamplesAmnt(train_ySTP, top_samples)*100)

        if iterationSTP[-1] > len(train_x) - num_initial_points:
            print("Maximum number of iterations exceeded, breaking loop.")
            return topSTP, iterationSTP
            break
            
    print(f'Iterations {iterationSTP[-1]}')
    return topSTP, iterationSTP

def runMaxSTP(seed):   
    print("Maximizing with STP")
    set_seeds(seed)
    iterationSTP = [0]
    topSTP = [0]

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.95)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y < top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesSTP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesSTP = non_top_5_indices[indicesSTP]

    # Select the corresponding points from train_x and train_y
    train_xSTP = train_x[indicesSTP]
    train_ySTP = train_y[indicesSTP]

    print(f"Seed: {seed}")
    print(f"Initial points: {train_xSTP}")
        
    while topSTP[-1] < 100:
        iterationSTP.append(iterationSTP[-1] + 1)
        # standardize the initial inputs and outputs before
        train_xSTP = TorchStd.transform(train_xSTP)
        train_ySTP = TorchStandardScaler().fit_transform(train_ySTP).flatten()

        # normalize it  
        train_xSTP = TorchNorm.transform(train_xSTP)
        train_ySTP = TorchNormalizer().fit_transform(train_ySTP).flatten()

        #def train_and_test_approximate_gp(objective_function_cls):
        model = STP(train_xSTP).to(dtype=torch.float64)
        #likelihood = gpytorch.likelihoods.GaussianLikelihood()
        likelihood = gpytorch.likelihoods.StudentTLikelihood()
        objective_function = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_ySTP.numel())
        # replace with gpytorch mll
        optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

        # Train
        model.train()
        #put it into training mode
        likelihood.train()

        for _ in range(50):
            output = model(train_xSTP)
            loss = -objective_function(output, train_ySTP)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Test
        model.eval()
        likelihood.eval()

        # works wtih the variational distribution
        with gpytorch.settings.num_likelihood_samples(512):
            pred = likelihood(model(TorchStd.transform(train_x))) # predict values for all candidates

        samples = pred.sample()
        # use as inputs to EI 
        meanSTP = samples.mean(dim=0)
        stdSTP = samples.std(dim=0)

        # Calculate the best observed value for EI
        best_observed_value = train_ySTP.max()
        
        # Select new points using EI
        acq_values = EI(meanSTP, stdSTP, best_observed_value, minimize=False)
        acq_values[indicesSTP] = -float('inf')
        indicesSTP = torch.cat([indicesSTP, acq_values.argmax().unsqueeze(0)])

        # Convert new_indices to a tensor
        # Add the new points to the training data
        train_xSTP = train_x[indicesSTP]
        train_ySTP = train_y[indicesSTP]
        
        topSTP.append(TopSamplesAmnt(train_ySTP, top_samples)*100)
        if iterationSTP[-1] > len(train_x) - num_initial_points:
            print("Maximum number of iterations exceeded, breaking loop.")
            print(f"Reached {topSTP[-1]}%")
            return topSTP, iterationSTP
            break

    print(f'Iterations {iterationSTP[-1]}')

    return topSTP, iterationSTP

def runMinEGP(seed):
    print("Minimizing with EGP")
    set_seeds(seed)    
    iterationEGP = [0]
    topEGP = [0]

    N = len(train_y) # Number of points you want to sample

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.05)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y > top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesEGP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesEGP = non_top_5_indices[indicesEGP]

    # Select the corresponding points from train_x and train_y
    train_xEGP = train_x[indicesEGP]
    train_yEGP = train_y[indicesEGP]

    print(f"Seed: {seed}")
    print(f"Initial points: {train_xEGP}")
    
    while topEGP[-1] < 100:
        iterationEGP.append(iterationEGP[-1] + 1)
        # Standardize the initial inputs and outputs
        train_xEGP = TorchStd.transform(train_xEGP)
        train_yEGP = TorchStandardScaler().fit_transform(train_yEGP).flatten()

        # normalize it  
        train_xEGP = TorchNorm.transform(train_xEGP)
        train_yEGP = TorchNormalizer().fit_transform(train_yEGP).flatten()

        # Optimize the model
        # Use a Half-Normal prior for the noise to find a Gaussian likelihood
        likelihood = GaussianLikelihood(noise_prior=gpytorch.priors.HalfNormalPrior(0.01))

        # Using the found likelihood, create a GP model
        gp = ExactGP(train_xEGP, train_yEGP, likelihood)
        mll = ExactMarginalLogLikelihood(likelihood, gp)

        # Fit the model by maximizing the marginal log likelihood
        gp.train()
        likelihood.train()
        
        try:
            fit_gpytorch_mll(mll)
        except RuntimeError as e:
            print(f"Optimization failed: {e}")
            break

        gp.eval()
        likelihood.eval()

        # predict from candidate pool
        with torch.no_grad():
            pred = gp(TorchStd.transform(train_x)) # predict values for all candidates

        meanEGP = pred.mean
        stdEGP = pred.stddev

        # Calculate the best observed value for EI
        best_observed_value = train_yEGP.min()
        
        # pass the predictions through an acquisition function to find the next best point to sample
        acq_values = EI(meanEGP, stdEGP, best_observed_value, minimize=True)
        acq_values[indicesEGP] = -float('inf')
        indicesEGP = torch.cat([indicesEGP, acq_values.argmax().unsqueeze(0)])
        

        # add the new point to the training data
        train_xEGP = train_x[indicesEGP]
        train_yEGP = train_y[indicesEGP]
        topEGP.append(TopSamplesAmnt(train_yEGP, top_samples)*100)
        
        if iterationEGP[-1] > len(train_x) - num_initial_points:
            print("Maximum number of iterations exceeded, breaking loop.")
            return topEGP, iterationEGP
            break

    print(f'Iterations {iterationEGP[-1]}')
    return topEGP, iterationEGP

def runMaxEGP(seed):
    print("Maximizing with EGP")
    set_seeds(seed)    
    iterationEGP = [0]
    topEGP = [0]
        # Assuming train_x and train_y are your datasets
    N = len(train_y) # Number of points you want to sample

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.95)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y < top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesEGP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesEGP = non_top_5_indices[indicesEGP]

    # Select the corresponding points from train_x and train_y
    train_xEGP = train_x[indicesEGP]
    train_yEGP = train_y[indicesEGP]

    print(f"Seed: {seed}")
    print(f"Initial points: {train_xEGP}")

    while topEGP[-1] < 100:
        iterationEGP.append(iterationEGP[-1] + 1)
        # standardize the initial inputs and outputs before
        # Standardize the initial inputs and outputs
        train_xEGP = TorchStd.transform(train_xEGP)
        train_yEGP = TorchStandardScaler().fit_transform(train_yEGP).flatten()

        # normalize it  
        train_xEGP = TorchNorm.transform(train_xEGP)
        train_yEGP = TorchNormalizer().fit_transform(train_yEGP).flatten()
        # optimize the model
        # use a half normal prior for the noise to find a Gaussian likelihood
        likelihood = GaussianLikelihood(noise_prior=gpytorch.priors.HalfNormalPrior(0.01))

        # using the found likelihood, create a GP model
        gp = ExactGP(train_xEGP, train_yEGP, likelihood)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp)

        # fit the model by maximizing the marginal log likelihood
        gp.train()
        likelihood.train()
        fit_gpytorch_mll(mll)

        gp.eval()
        likelihood.eval()

        # predict from candidate pool
        with torch.no_grad():
            pred = gp(TorchStd.transform(train_x)) # predict values for all candidates

        meanEGP = pred.mean
        stdEGP = pred.stddev

        # Calculate the best observed value for EI
        best_observed_value = train_yEGP.max()
    
        # pass the predictions through an acquisition function to find the next best point to sample
        acq_values = EI(meanEGP, stdEGP, best_observed_value, minimize=False)
        acq_values[indicesEGP] = -float('inf')
        indicesEGP = torch.cat([indicesEGP, acq_values.argmax().unsqueeze(0)])
        
        # add the new point to the training data
        train_xEGP = train_x[indicesEGP]
        train_yEGP = train_y[indicesEGP]
        topEGP.append(TopSamplesAmnt(train_yEGP, top_samples)*100)
        
        if iterationEGP[-1] > len(train_x) - num_initial_points:
            print(f"Reached {topEGP[-1]}%")
            print("Maximum number of iterations exceeded, breaking loop.")
            return topEGP, iterationEGP
            break

    print(f'Iterations {iterationEGP[-1]}')

    return topEGP, iterationEGP

def runMinVGP(seed):    
    print("Minimizing with VGP")
    set_seeds(seed)
    iterationVGP = [0]
    topVGP = [0]

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.05)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y > top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesVGP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesVGP = non_top_5_indices[indicesVGP]

    # Select the corresponding points from train_x and train_y
    train_xVGP = train_x[indicesVGP]
    train_yVGP = train_y[indicesVGP]

    print(f"Seed: {seed}")
    print(f"Initial points: {train_xVGP}")
    
    while topVGP[-1] < 100:
        
        iterationVGP.append(iterationVGP[-1] + 1)
        # standardize the initial inputs and outputs before

        train_xVGP = TorchStd.transform(train_xVGP)
        train_yVGP = TorchStandardScaler().fit_transform(train_yVGP).flatten()

        # normalize it  
        train_xVGP = TorchNorm.transform(train_xVGP)
        train_yVGP = TorchNormalizer().fit_transform(train_yVGP).flatten()

        #def train_and_test_approximate_gp(objective_function_cls):
        model = VariationalGP(train_xVGP).to(dtype=torch.float64)

        likelihood = GaussianLikelihood()
        objective_function = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_yVGP.numel())
        # replace with gpytorch mll
        optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

        # Train
        model.train()
        #put it into training mode
        likelihood.train()

        for _ in range(50):
            output = model(train_xVGP)
            loss = -objective_function(output, train_yVGP)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        # Test
        model.eval()
        likelihood.eval()

        # works wtih the variational distribution
        with torch.no_grad(), gpytorch.settings.num_likelihood_samples(512):
            pred = likelihood(model(TorchStd.transform(train_x))) # predict values for all candidates

        samples = pred.sample()
        # get the mean and standard deviation of the samples
        meanVGP = samples.mean(dim=0)
        stdVGP = samples.stddev(dim=0)
        
        # Calculate the best observed value for EI
        best_observed_value = train_yVGP.min()
        
        # Select new points using EI
        acq_values = EI(meanVGP, stdVGP, best_observed_value, minimize=True)
        acq_values[indicesVGP] = -float('inf')
        indicesVGP = torch.cat([indicesVGP, acq_values.argmax().unsqueeze(0)])

        # Convert new_indices to a tensor
        # Add the new points to the training data
        train_xVGP = train_x[indicesVGP]
        train_yVGP = train_y[indicesVGP]
        topVGP.append(TopSamplesAmnt(train_yVGP, top_samples)*100)

        if iterationVGP[-1] > len(train_x) - num_initial_points:
            print("Maximum number of iterations exceeded, breaking loop.")
            print(f"Reached {topVGP[-1]}%")
            return topVGP, iterationVGP
            break

    print(f'Iterations {iterationVGP[-1]}')

    return topVGP, iterationVGP
    
def runMaxVGP(seed):   
    print("Maximizing with VGP") 
    set_seeds(seed)
    iterationVGP = [0]
    topVGP = [0]

    # Calculate the threshold for the top 5%
    top_5_percent_threshold = torch.quantile(train_y, 0.95)

    # Identify indices of samples not in the top 5%
    non_top_5_indices = (train_y < top_5_percent_threshold).nonzero(as_tuple=True)[0]

    # Randomly sample from these non-top-5% indices
    indicesVGP = torch.randperm(len(non_top_5_indices))[:num_initial_points]
    indicesVGP = non_top_5_indices[indicesVGP]

    # Select the corresponding points from train_x and train_y
    train_xVGP = train_x[indicesVGP]
    train_yVGP = train_y[indicesVGP]
    print(f"Seed: {seed}")
    print(f"Initial points: {train_xVGP}")
    
    while topVGP[-1] < 100:
        
        iterationVGP.append(iterationVGP[-1] + 1)
        # standardize the initial inputs and outputs before
        train_xVGP = TorchStd.transform(train_xVGP)
        train_yVGP = TorchStandardScaler().fit_transform(train_yVGP).flatten()

        # normalize it  
        train_xVGP = TorchNorm.transform(train_xVGP)
        train_yVGP = TorchNormalizer().transform(train_yVGP).flatten()

        #def train_and_test_approximate_gp(objective_function_cls):
        model = VariationalGP(train_xVGP).to(dtype=torch.float64)

        likelihood = GaussianLikelihood()
        objective_function = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_yVGP.numel())
        # replace with gpytorch mll
        optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

        # Train
        model.train()
        #put it into training mode
        likelihood.train()

        for _ in range(50):
            output = model(train_xVGP)
            loss = -objective_function(output, train_yVGP)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        # Test
        model.eval()
        likelihood.eval()

        # predict from candidate pool
        with torch.no_grad(), gpytorch.settings.num_likelihood_samples(512):
            pred = likelihood(model(TorchStd.transform(train_x))) # predict values for all candidates

        # find the mean and standard deviation of the preidictions
        meanVGP = pred.mean
        stdVGP = pred.stddev

        # Calculate the best observed value for EI
        best_observed_value = train_yVGP.max()
      
        # Select new points using EI
        acq_values = EI(meanVGP, stdVGP, best_observed_value, minimize=False)
        acq_values[indicesVGP] = -float('inf')
        indicesVGP = torch.cat([indicesVGP, acq_values.argmax().unsqueeze(0)])

        # Convert new_indices to a tensor
        # Add the new points to the training data
        train_xVGP = train_x[indicesVGP]
        train_yVGP = train_y[indicesVGP]
        topVGP.append(TopSamplesAmnt(train_yVGP, top_samples)*100)

        if iterationVGP[-1] > len(train_x) - num_initial_points:
            print("Maximum number of iterations exceeded, breaking loop.")
            print(f"Reached {topVGP[-1]}%")
            return topVGP, iterationVGP
            break

    print(f'Iterations {iterationVGP[-1]}')

    return topVGP, iterationVGP


# Define your classes and functions here...
# (Keep your STP, ExactGP, VariationalGP, TorchStandardScaler, TorchNormalizer, EI, and other functions as they are)

# Parallelization function for each campaign
def run_campaign(seed, max_or_min):
    if max_or_min == "max":
        topEGP, iterationEGP = runMaxEGP(seed)
        topSTP, iterationSTP = runMaxSTP(seed)
        topVGP, iterationVGP = runMaxVGP(seed)
    else:
        topEGP, iterationEGP = runMinEGP(seed)
        topSTP, iterationSTP = runMinSTP(seed)
        topVGP, iterationVGP = runMinVGP(seed)
    
    return (topEGP, iterationEGP), (topSTP, iterationSTP), (topVGP, iterationVGP)

# Initialize datasets and common variables
datasets = [("P3HT", "max"), ("Perovskite", "min")]
campaigns = 50
num_initial_points = 3
num_new_samples_per_iteration = 1
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import gpytorch
import math
import seaborn as sns
import random
from tqdm import trange, tqdm
from collections import Counter
from scipy.interpolate import splrep, interp1d
from sklearn import preprocessing
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.likelihoods import GaussianLikelihood, StudentTLikelihood
from gpytorch.models import ApproximateGP
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.means import ConstantMean
from gpytorch.mlls import ExactMarginalLogLikelihood, VariationalELBO
import matplotlib.ticker as ticker
import matplotlib.tri as tri
import matplotlib.font_manager as font_manager
from torch.multiprocessing import Pool, set_start_method
from sklearn.preprocessing import StandardScaler
import time

# Check for MPS and CUDA device availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device found, setting as device.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA device found, setting as device.")
else:
    device = torch.device("cpu")
    print("Neither MPS nor CUDA device found. Using default device (CPU).")

# Set the seed for all random use
def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)

# Define your classes and functions here...
# (Keep your STP, ExactGP, VariationalGP, TorchStandardScaler, TorchNormalizer, EI, and other functions as they are)

# Parallelization function for each campaign
def run_campaign(args):
    seed, max_or_min = args
    if max_or_min == "max":
        topEGP, iterationEGP = runMaxEGP(seed)
        topSTP, iterationSTP = runMaxSTP(seed)
        topVGP, iterationVGP = runMaxVGP(seed)
    else:
        topEGP, iterationEGP = runMinEGP(seed)
        topSTP, iterationSTP = runMinSTP(seed)
        topVGP, iterationVGP = runMinVGP(seed)
    
    return (topEGP, iterationEGP), (topSTP, iterationSTP), (topVGP, iterationVGP)

# Initialize datasets and common variables
datasets = [("P3HT", "max"), ("Perovskite", "min")]
campaigns = 50

for element, max_or_min in datasets:
    print(f"Starting {element}")

    data = pd.read_csv(f"datasets/{element}_dataset.csv")
    data = data.groupby(data.columns[-1]).mean().reset_index()
    train_x = torch.tensor(data.iloc[:, 1:].values, dtype=torch.float)
    train_y = torch.tensor(data.iloc[:, 0].values, dtype=torch.float).unsqueeze(1)

    N = len(train_x)

    # We are using predefined candidates, so we can scale at the start
    TorchStd = TorchStandardScaler()
    TorchStd.fit(train_x)
    TorchNorm = TorchNormalizer()
    TorchNorm.fit(train_x)

    total_samples = len(train_y)

    set_seeds(22)

    n_top = int(math.ceil(N * 0.05))

    # Find the top 5% of the samples
    train_y_df = pd.DataFrame(train_y.numpy(), columns=[0])
    if max_or_min == "max":
        top_samples = train_y_df.nlargest(n_top, train_y_df.columns[0], keep='first').iloc[:, 0].values.tolist()
    else:
        top_samples = train_y_df.nsmallest(n_top, train_y_df.columns[0], keep='first').iloc[:, 0].values.tolist()

    print(f"Number of of top 5% samples: {len(top_samples)}")
    print(f"Top 5% samples: {top_samples}")

    def TopSamplesAmnt(y, top_samples):
        return len([i for i in y if i in top_samples]) / len(top_samples)

    # Generate a list of seeds randomly picked from the range 0-1000 equal to the number of campaigns without repeating
    seedList = random.sample(range(1000), campaigns)

    # Prepare arguments for parallel execution
    args = [(seed, max_or_min) for seed in seedList]

    # Run campaigns in parallel with progress bar
    start_time = time.time()
    with Pool() as pool:
        results = list(tqdm(pool.imap(run_campaign, args), total=campaigns, desc=f"Processing {element} campaigns"))
    end_time = time.time()

    # Print the time taken
    print(f"Time taken for {element} campaigns: {end_time - start_time:.2f} seconds")

    # Process the results
    topEGP_results = [result[0] for result in results]
    topSTP_results = [result[1] for result in results]
    topVGP_results = [result[2] for result in results]

    # Function to dynamically collect arrays
    def collect_arrays(results, index):
        arrays = []
        for result in results:
            arrays.append(result[index])
        return arrays

    # Function to pad arrays with the last element to match the maximum length
    def pad_array(array, max_length):
        return np.pad(array, (0, max_length - len(array)), 'constant', constant_values=array[-1])

    def find_max_length(results, index):
        arrays = collect_arrays(results, index)
        return max(len(arr[0]) for arr in arrays)

    # Process arrays for each type
    def process_arrays(results, index, max_length):
        arrays = collect_arrays(results, index)
        padded_arrays = [pad_array(arr[0], max_length) for arr in arrays]
        stack = np.stack(padded_arrays)
        mean_values = np.mean(stack, axis=0)
        std_values = np.std(stack, axis=0)
        return mean_values, std_values

    # Process arrays for STP, EGP, and VGP
    max_length_STP = find_max_length(topSTP_results, 0)
    max_length_EGP = find_max_length(topEGP_results, 0)
    max_length_VGP = find_max_length(topVGP_results, 0)
    max_length = max(max_length_STP, max_length_EGP, max_length_VGP)
    mean_topSTP, std_valuesSTP = process_arrays(topSTP_results, 0, max_length)
    mean_topEGP, std_valuesEGP = process_arrays(topEGP_results, 0, max_length)
    mean_topVGP, std_valuesVGP = process_arrays(topVGP_results, 0, max_length)

    # Ensure that the number of iterations matches the longest array length
    iterations = np.arange(1, max_length + 1)

    sns.set(style="whitegrid")
    # Plot the mean and fill between the min and max for each type
    fig = plt.figure(figsize=(16, 12), dpi=800)
    ax = plt.subplot(111)

    # Plot for STP
    ax.plot(iterations, mean_topSTP, label='Mean STP', color='blue', linewidth=2)
    ax.fill_between(iterations, mean_topSTP - std_valuesSTP, mean_topSTP + std_valuesSTP, label='STP Standard Deviation', color='blue', alpha=0.1)

    # Plot for EGP
    ax.plot(iterations, mean_topEGP, label='Mean EGP', color='orange', linewidth=2)
    ax.fill_between(iterations, mean_topEGP - std_valuesEGP, mean_topEGP + std_valuesEGP, label='EGP Standard Deviation', color='orange', alpha=0.1)

    # Plot for VGP
    ax.plot(iterations, mean_topVGP, label='Mean VGP', color='green', linewidth=2)
    ax.fill_between(iterations, mean_topVGP - std_valuesVGP, mean_topVGP + std_valuesVGP, label='VGP Standard Deviation', color='green', alpha=0.1)

    plt.xlabel('Number of Iterations', fontsize=25)
    plt.ylabel('Percentage of Top 5% Samples Found', fontsize=25)
    plt.title(f'Average Percentage of Top Samples from the {element} Dataset \n Found Over {campaigns} Campaigns Using EI', fontsize=30, pad=20)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=20)

    # Customize ticks
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    # Adjust axis limits and aspect ratio if needed
    plt.ylim(0, 100)
    plt.xlim(0, max_length)

    # Save the plot
    plt.savefig(f"EI_{element}_{campaigns}_Campaigns_{max_or_min.capitalize()}.png")
    plt.show()


CUDA device found, setting as device.
Starting P3HT
Number of of top 5% samples: 12
Top 5% samples: [1243.6700439453125, 1089.1199951171875, 904.2899780273438, 852.3300170898438, 839.280029296875, 838.3099975585938, 824.469970703125, 804.1099853515625, 800.5800170898438, 788.8326416015625, 772.9400024414062, 770.3499755859375]


Processing P3HT campaigns:   0%|          | 0/50 [00:00<?, ?it/s]